In [1]:
from random import randint

def CalcRoots(F, p, R, g, roots):
    while True:
       # print("g: ", g)
        delta = randint(0, p)
        d = R.gcd((x + delta)**((p-1)/2) - 1, g)
       # print("d: ", d)
        if d == 1 or d == g:
            if g.degree() == 1 and d == g:
                roots.append(d.coefficients()[0]*-1)
                return roots
            continue
        if d.degree() == 1:
            roots.append(d.coefficients()[0]*-1)
            g = R(g/d)
            continue
        if d.degree() == g.degree() - 1:
            d2 = R(g/d)
            roots.append(d2.coefficients()[0]*-1)
            g = d
            continue
        if 2 <= d.degree() and d.degree() < g.degree() - 1:
            d2 = R(g/d)
            CalcRootsRec(F, p, R, d, roots)
            CalcRootsRec(F, p, R, d2, roots)
            break
        return roots

def first_alg(f, f_test, R, p,roots):
    res=R.gcd(f_test,f)
    if res.degree() == 0:
        roots.append('No res')
        return 
    elif res.degree( ) == 1:
        roots.append(res.coefficients()[0]*-1)
        return
    elif res.degree() == p:
        roots.append('All elements')
        return
    CalcRoots(f_test, p, R, res, roots)
    return roots

p = 21287447

p = 3380659
p = 1241587
#p=3**9
F=GF(p)

R.<x> = PolynomialRing(F)

f = R(x**p - x)
f_1 = R(x**18+1576544*x**17+1269679*x**16+421388*x**15+420073*x**14+93274*x**13+419461*x**12
       +1552845*x**11+268691*x**10+1564101*x**9+593897*x**8+1360879*x**7+2039035*x**6+
       779162*x**5+857188*x**4+1791605*x**3+1708777*x**2+27962*x+1688424)


f_1 = R(x**18 +1241555*x**17 +378*x**16 +1240134*x**15 +
      1232709*x**14+113875*x**13 +872270*x**12 +574774*x**11 +59335*x**10 +306073*x**9 +
      306382*x**8 + 349756*x**7 + 892229*x**6 +192842*x**5 + 789246*x**4 +155738*x**3 +
      363914*x**2 +238698*x+981554)
f_1 = R(x**17 + 3371938*x**15 + 2656364*x**14 + 
      839900*x**13 + 189567*x**12 + 562171*x**11 + 59966*x**10 + 
      183633*x**9 + 916975*x**8 + 249259*x**7 + 55393*x**6 + 
      467855*x**5 + 1959961*x**4 + 1710912*x**3 + 2461019*x**2 + 
      1848930*x + 831453)
#print(f_1 in R)
print('sage roots:', f_1.roots())
roots=[]
res=first_alg(f, f_1,R, p,roots)
if roots[0]=='No res' or roots[0]=='All elements':
    print('Our roots:')
    print(roots[0])
else:
    #res=first_alg(f, f_2,R, q, p,roots)
    #print('roots:',roots)
    roots_res=roots
    for i in range(len(roots)):
        root=roots[i]
        f_new=f_1
        f_root=R(x-int(root))
        r=R(f_new/f_root)
        while R(r%f_root)==R(0):
            roots_res.append(root)
            f_new=R(f_new/f_root)
            r1=R(f_new/f_root)

    print('Our roots:')
    for root in roots:
        print('root: '+str(root)+' multiplicity: '+str(roots_res.count(root)))

#print(f_1(x=roots[1]))
#print(f_1(x=roots[0]))



sage roots: [(2*z9^6 + 2*z9^4 + z9^2 + z9, 1), (2*z9^6 + 2*z9^4 + z9^2 + z9 + 2, 1), (2*z9^6 + 2*z9^4 + z9^2 + z9 + 1, 1), (0, 2)]


KeyboardInterrupt: 

In [10]:
import numbers
import math
import numpy as np
from scipy.linalg import lu
import sympy
import sys

def convertToPoly(H,K,U):
    r=len(H)
    for i in range(r):
        result=0
        for j in range(len(H[i])):
            result+=H[i][j] * K(x**j)
        U.append(result)
    #print(res)




def berli_kemp(f,new_diff,new_gcd,K,q):
    n=f.degree()
   
    b_polynomials = []
    for i in range(0, n):
        b_polynomials.append(K(K.gen()**(q*i) % f))
    #print(b_polynomials)
    b_coefficients = []
    for b_polynomial in b_polynomials:
        coefficients = [0] * n
        for monomial in b_polynomial.monomials():
            coefficients[monomial.degree()]  =b_polynomial.monomial_coefficient(monomial)
        b_coefficients.append(coefficients)
    res=matrix(b_coefficients)
    I = matrix.identity(n)
    B_I_T = (res - I)
    rank = B_I_T.rank()
    if rank == n - 1:
        return -1
    #print(rang)
  #  print(B_I_T)
   # print(B_I_T.kernel())
    solve_basis = B_I_T.left_kernel().basis()
    r = len(solve_basis)
    l,i,U = 1,2,[f]
    solnPolys=[]
    convertToPoly(solve_basis,K,solnPolys)
    #print(solnPolys)
    while l!=r and i!=r+1:
        h=solnPolys[i-1]
        #print(h)
       # print('r:',r)
       # print('l:',l)
        for j in range(1,l+1):
            res=[]
            for k in range(q):
                result = K.gcd(U[j-1],K(h-k))
                if result.degree():
                    res.append(result)
            U[j-1]=res[0]
            for t in range(2,len(res)+1):
                U.append(res[t-1])
                #print(U)
            l=l+len(res)-1
            if l==r:
                return U
                
           # print('res:',res)
        i+=1
            
def preporation(f2,Kx,p):
    factorization=[]
    g=Kx.gcd(f2,diff(f2))
    flag_diff=0
    while 1:
        while diff(f2)==0:
            g=Kx(0)
            flag_diff=flag_diff+p
            for i in range(f2.degree()+1):
                g=g+int(f2[i])*Kx(x**(i//p))
            #print("Извлечение корня\ng = "+str(g)+" g^p = "+str(g**p))
            f2=g
        g=Kx.gcd(f2,diff(f2))
        while g.degree():
            while diff(g)==0:
                g1=Kx(0)
                for i in range(g.degree()+1):
                    g1=g1+g[i]*Kx(x**(i//p))
                g=g1
            #print("НОД = {}".format(g))
            while f2%g==0:
                factorization.append(g)
                f2=f2//g
            g=Kx.gcd(f2,diff(f2))
        if diff(f2)!=0 and g.degree()==0:
            break
  
    #print(factorization)
   # print('f2 ',f2,'diff ',diff(f2))
    return factorization,flag_diff,f2    
    
def start_work(f_2, K,q):
    result_x = 1
    flag_dif = 0
    factorization = []
    if diff(f_2) == 0:
        result_x = q
        g = K.random_element((f_2.degree() // q, f_2.degree() // q))
        while diff(f_2) == 0:
            print(diff(f_2))
            if diff(g) != 0 and diff(f_2 // g) != 0 and g^q == f_2:
                flag_dif=flag_dif+q
                f_2 = g
                continue
            g = K.random_element((f_2.degree() // q, f_2.degree() // q))

    g = K.gcd(f_2, diff(f_2))
    while g.degree():
        factorization.append(g)
        f_2 = f_2 // g
        g = K.gcd(f_2, diff(f_2))
    print(f_2)
    print(factorization)
    return factorization, f_2, result_x,flag_dif

#q = 1922603
#q=3
#q=2
q=2
Z=Zmod(q)
#print(q)
K=PolynomialRing(Z, 'x')

#print(K)
f_2 = K(x**18+7288*x**17+1569197*x**16+462295*x**15+832234*x**14+72126*x**13+410429*x**12
       +1874129*x**11+531482*x**10+1741564*x**9+1632964*x**8+1066504*x**7+1105905*x**6+
       1062657*x**5+417643*x**4+359234*x**3+131984*x**2+220096*x+66760)
f_2=K(x^24+x^16+x^12+x^8+x^4+1)
#f_2=K((x + 1)^2 * (x^4 + x + 1)^4)
#f_2 = K(x**27+x**18+x**12+x**6+1)
#f_2=K(x**4+x**2+x+1)
#f_2 = K(3 + 2*x + 3*x**2 + 3*x**3+3*x**4+3*x**5)
#f_2=K(1+x+x**3+x**4+x**7+x**8)
#f_2=K((x + 1)^6 * (x^3 + 2*x^2 + 2*x + 2)^3 * (x^4 + 2*x^3 + 2)^3)
#f_2=K((x + 1)^5 * (x^3 + 2*x^2 + 2*x + 2)^3 * (x^4 + 2*x^3 + 2)^3)
#print(f_2)
print('sage :', f_2.factor())
factorization=[]
factorization,flag_dif,f_2=preporation(f_2,K,q)
#print(g)
#print(factorization)
#print('f_2:',f_2)

U=berli_kemp(f_2,diff(f_2),K.gcd(f_2, diff(f_2)),K,q)
res=[]
if U==-1:
    U=[]
    U.append(f_2)

for fact in factorization:
   # r=str(fact)
    #print(r)
    u=berli_kemp(fact,diff(fact),K.gcd(fact, diff(fact)),K,q)

   # print('u:',u)
    if u!=-1:
        U.append(u)
    else:
        U.append(fact)
factorization = []

for u in U:
    if flag_dif==0:
        root = {'root': u, 'multiplicity': U.count(u)}
    else:
        root = {'root': u, 'multiplicity': U.count(u)*flag_dif}
    if root not in factorization:
        factorization.append(root)
print('Our roots:')
print(factorization)

print("end")


sage : (x + 1)^8 * (x^4 + x + 1)^4
Our roots:
[{'root': x^4 + x + 1, 'multiplicity': 4}, {'root': x + 1, 'multiplicity': 8}]
end


In [15]:
q = 1922603
q=3
Z=Zmod(q)
K=PolynomialRing(Z, 'x')
f_2 = K(x**18+7288*x**17+1569197*x**16+462295*x**15+832234*x**14+72126*x**13+410429*x**12
       +1874129*x**11+531482*x**10+1741564*x**9+1632964*x**8+1066504*x**7+1105905*x**6+
       1062657*x**5+417643*x**4+359234*x**3+131984*x**2+220096*x+66760)
#f_2 = K(x**27+x**18+x**12+x**6+1)
#res=f_2/K(x^2 + 1211645*x + 1671040)/K(x^10 + 202735*x^9 + 1189133*x^8 + 1628698*x^7 + 583880*x^6 + 885089*x^5 + 436494*x^4 + 19925*x^3 + 411175*x^2 + 963996*x + 1494288)/K(x^5 + 1594599*x^4 + 855129*x^3 + 68687*x^2 + 703616*x + 823895)
f_2=K(((x^6 + 2*x^3 + 1)*( x^9 + 2*x^6 + 2*x^3 + 2)* (x^12 + 2*x^9 + 2)))

print(f_2)

x^27 + x^18 + x^12 + x^6 + 1
